In [1]:
import pandas as pd
import numpy as np
import pyarrow 
import seaborn as sb
import requests
from bs4 import BeautifulSoup as bs
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager
import time

In [2]:
url_hoteles = "https://all.accor.com/ssr/app/ibis/hotels/madrid-spain/index.es.shtml?compositions=1&stayplus=false&snu=false&hideWDR=false&accessibleRooms=false&hideHotelDetails=false&dateIn=2025-03-01&nights=1&destination=madrid-spain"

In [3]:
respuesta_hoteles = requests.get(url_hoteles)
respuesta_hoteles.status_code

200

In [4]:
sopa_hoteles = bs(respuesta_hoteles.content, "html.parser")

In [5]:
hoteles = sopa_hoteles.find_all("a", class_="title__link")[:10]
len(hoteles)

10

In [6]:
dictio_scrap = {
    "nombre_hotel": [],
    "estrellas": [],
    "precio_noche": []}

In [7]:
for hotel in hoteles:
    #Nombre del hotel
    nombre = hotel.find("h2")
    nombre_hotel = nombre.get_text(strip=True) if nombre else "Sin nombre"

    #Estrsellas
    estrellas = hotel.find("span", class_="sr-only") #Buscar el número de estrellas
    estrellas_hotel = estrellas.get_text(strip=True) if estrellas else "Sin estrellas"

    precio = hotel.find("span", class_="booking-price__number mcp-price-number") #Buscar el precio del hotel
    precio_noche = precio.get_text(strip=True) if precio else "Sin precio"

    dictio_scrap["nombre_hotel"].append(nombre_hotel)
    dictio_scrap["estrellas"].append(estrellas_hotel)
    dictio_scrap["precio_noche"].append(precio_noche)

In [8]:
def scrap_info_hoteles(url):
# Diccionario para almacenar los datos
    dictio_scrap = {
        "nombre_hotel": [],
        "estrellas": [],
        "precio_noche": []  # ✅ Agregamos una lista para el precio
    }
    # URL de ejemplo (ajusta según la web que estás scrapeando)
    res = requests.get(url)

    if res.status_code == 200:
        sopa = bs(res.content, "html.parser")

        # Encontrar todos los hoteles en la página
        hoteles = sopa.find_all("a", class_="title__link")[:10]

        for hotel in hoteles:
            # ✅ Nombre del hotel
            nombre_hotel = hotel.contents[0].strip() if hotel else "Sin nombre"

            # ✅ Estrellas del hotel
            estrellas = hotel.find("span", class_="sr-only")
            estrellas_hotel = estrellas.get_text(strip=True) if estrellas else "Sin estrellas"

            # ✅ Precio por noche
            precio = hotel.find("span", class_="booking-price__number mcp-price-number")
            precio_noche = precio.get_text(strip=True) if precio else "Sin precio"

            # ✅ Agregar cada dato en su lista correspondiente
            dictio_scrap["nombre_hotel"].append(nombre_hotel)
            dictio_scrap["estrellas"].append(estrellas_hotel)
            dictio_scrap["precio_noche"].append(precio_noche)  # ✅ Ahora el precio va en la lista correcta
    return dictio_scrap

In [9]:
url = "https://all.accor.com/ssr/app/ibis/hotels/madrid-spain/index.es.shtml?compositions=1&stayplus=false&snu=false&hideWDR=false&accessibleRooms=false&hideHotelDetails=false&dateIn=2025-03-01&nights=1&destination=madrid-spain"

In [10]:
dictio_hoteles_competencia = scrap_info_hoteles(url)
df_hoteles_competencia = pd.DataFrame(dictio_hoteles_competencia)
df_hoteles_competencia

,nombre_hotel,estrellas,precio_noche
0,ibis Styles Madrid Prado,3 Estrellas,Sin precio
1,Mercure Madrid Centro,4 Estrellas,Sin precio
2,ibis budget Madrid Centro Lavapies,1 Estrellas,Sin precio
3,ibis Styles Madrid Centro Maravillas,2 Estrellas,Sin precio
4,Novotel Madrid Center,4 Estrellas,Sin precio
5,ibis budget Madrid Calle 30,1 Estrellas,Sin precio
6,ibis Madrid Centro las Ventas,2 Estrellas,Sin precio
7,ibis budget Madrid Centro las Ventas,1 Estrellas,Sin precio
8,Novotel Madrid City Las Ventas,4 Estrellas,Sin precio
9,ibis Styles Madrid City Las Ventas,4 Estrellas,Sin precio


In [11]:
#Eliminamos la palabra "estrellas" de la columna estrellas para dejarlo en formato int:

df_hoteles_competencia["estrellas"] = df_hoteles_competencia["estrellas"].str.replace(" Estrellas", "", regex = False).astype(int)

In [12]:
df_hoteles_competencia.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10 entries, 0 to 9
Data columns (total 3 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   nombre_hotel  10 non-null     object
 1   estrellas     10 non-null     int64 
 2   precio_noche  10 non-null     object
dtypes: int64(1), object(2)
memory usage: 372.0+ bytes


Utilizamos selenium para sacar los datos del precio de cada noche

In [19]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager

def scrap_info_hoteles(url):
    """
    Extrae información de hoteles desde la web de Accor utilizando Selenium.
    
    Parámetros:
    - url (str): URL de la página de hoteles.

    Retorna:
    - dict: Un diccionario con los datos extraídos:
        - "nombre_hotel": Lista con los nombres de los hoteles.
        - "estrellas": Lista con la clasificación en estrellas de cada hotel.
        - "precio_noche": Lista con el precio por noche.
    """
    # Configuración de Selenium (headless opcional)
    options = Options()
    options.add_argument("--start-maximized")  # Maximizar ventana para evitar errores de carga
    # options.add_argument("--headless")  # Descomenta para ejecutar sin interfaz gráfica
    
    # Inicializar WebDriver
    service = Service(ChromeDriverManager().install())
    driver = webdriver.Chrome(service=service, options=options)

    # Diccionario para almacenar los datos
    dictio_scrap = {
        "nombre_hotel": [],
        "estrellas": [],
        "precio_noche": []
    }

    # Cargar la URL
    driver.get(url)

    # ✅ Esperar a que los elementos de los hoteles aparezcan en la página
    info_hoteles = WebDriverWait(driver, 10).until(
        EC.presence_of_all_elements_located((By.CLASS_NAME, "hotelblock__content"))
    )

    # ✅ Recorrer los primeros 10 hoteles y extraer datos
    for hotel in info_hoteles[:10]:
        try:
            # Extraer nombre del hotel
            nombre_hotel = hotel.find_element(By.CLASS_NAME, "title__link").text.split("\n")[0]

            # Extraer estrellas del hotel
            estrellas_hotel = hotel.find_element(By.CLASS_NAME, "title__link").text.split("\n")[1].split(" ")[0]

            # Guardar en el diccionario
            dictio_scrap["nombre_hotel"].append(nombre_hotel)
            dictio_scrap["estrellas"].append(estrellas_hotel)
        except Exception as e:
            print(f"Error al obtener nombre o estrellas: {e}")
            dictio_scrap["nombre_hotel"].append("Sin nombre")
            dictio_scrap["estrellas"].append("Sin estrellas")

    # ✅ Esperar los precios
    precios_hoteles = WebDriverWait(driver, 10).until(
        EC.presence_of_all_elements_located((By.CLASS_NAME, "rate-details__price-wrapper"))
    )

    # ✅ Extraer precios de los primeros 10 hoteles
    for precio_hotel in precios_hoteles[:10]:
        try:
            precio = precio_hotel.text.split("\n")[1].replace("€", "").strip()
            dictio_scrap["precio_noche"].append(precio)
        except Exception as e:
            print(f"Error al obtener precio: {e}")
            dictio_scrap["precio_noche"].append("Sin precio")

    # ✅ Cerrar el navegador
    driver.quit()
    
    return dictio_scrap

In [21]:
# URL de la web con hoteles en Madrid
url_hoteles = "https://all.accor.com/ssr/app/ibis/hotels/madrid-spain/index.es.shtml?dateIn=2025-03-01&nights=1"

# Ejecutar el scraping
datos_hoteles = scrap_info_hoteles(url_hoteles)

# Convertir a DataFrame y visualizar
df_hoteles_competencia = pd.DataFrame(datos_hoteles)
df_hoteles_competencia

,nombre_hotel,estrellas,precio_noche
0,ibis Styles Madrid Prado,3,179
1,Mercure Madrid Centro,4,172
2,ibis budget Madrid Centro Lavapies,1,119
3,ibis Styles Madrid Centro Maravillas,2,95
4,Novotel Madrid Center,4,116
5,ibis budget Madrid Calle 30,1,90
6,ibis Madrid Centro las Ventas,2,97
7,ibis budget Madrid Centro las Ventas,1,83
8,Novotel Madrid City Las Ventas,4,99
9,ibis Styles Madrid City Las Ventas,4,77


In [26]:
df_hoteles_competencia["estrellas"] = df_hoteles_competencia["estrellas"].astype(int)
df_hoteles_competencia.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10 entries, 0 to 9
Data columns (total 3 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   nombre_hotel  10 non-null     object
 1   estrellas     10 non-null     int64 
 2   precio_noche  10 non-null     object
dtypes: int64(1), object(2)
memory usage: 372.0+ bytes


In [27]:
df_hoteles_competencia.to_csv("../data/10_hoteles_competencia.csv", index=False)